## Analyze
Merge our data sources and compare the spatial locations.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS geodata.gold;

DROP TABLE IF EXISTS geodata.gold.well_header;

## Union the tables
Combine the three vendor tables into a single `well_header` table. The combined count should be ~28000 in 2025.

In [0]:
occ_df = spark.table("geodata.silver.well_header_occ")
sp_df = spark.table("geodata.silver.well_header_sp")
env_df = spark.table("geodata.silver.well_header_env")

wh_df = occ_df.unionByName(sp_df).unionByName(env_df)
wh_df.write.format("delta").saveAsTable("geodata.gold.well_header")

In [0]:
%sql
SHOW TABLES IN geodata.gold;


In [0]:
%sql
select count(*) as three_vendor_count from geodata.gold.well_header

## Calculate the spatial distances
Finally, we can compare the surface locations. For expediency, we'll only compare wells that are avaliable from all three sources. We use `ST_DISTANCESPHEROID` to calculate the distance (in Meters not degrees) between sets of two points. The output looks something like this:

| ... | distance_occ_env | distance_env_sp | distance_occ_sp | ... |
| -- | -- | -- | -- | -- |
| ... | 11.11 | 22.22 | 33.33 | ... |
| ... | 44.44 | 55.55 | 66.66 | ... |

...where three `distance_` columns express the difference between all sets of two points.

In [0]:
from pyspark.sql.functions import col, when, expr

well_header_df = spark.table("geodata.gold.well_header")

enverus_well_name = (
    well_header_df
    .filter(col("vendor") == "ENV")
    .select("uwi_10", "well_name")
    .withColumnRenamed("well_name", "well_name_ENV")
)

# Pivot the table wide by vendor:
wide_df = (
    well_header_df
    .groupBy("uwi_10")
    .pivot("vendor", ["ENV", "OCC", "SP"])
    .agg(expr("first(geom)"))
    .withColumnRenamed("ENV", "geom_ENV")
    .withColumnRenamed("OCC", "geom_OCC")
    .withColumnRenamed("SP",  "geom_SP")
)

# Compute distances for all available pairs
wide_df = wide_df.withColumn(
    "distance_env_occ",
    expr("ST_DISTANCESPHEROID(geom_ENV, geom_OCC)")
).withColumn(
    "distance_env_sp",
    expr("ST_DISTANCESPHEROID(geom_ENV, geom_SP)")
).withColumn(
    "distance_occ_sp",
    expr("ST_DISTANCESPHEROID(geom_OCC, geom_SP)")
)

# Filter for at least 2 non-null geoms if desired
wide_df = wide_df.filter(
    (
        (col("geom_ENV").isNotNull() & col("geom_OCC").isNotNull()) |
        (col("geom_ENV").isNotNull() & col("geom_SP").isNotNull()) |
        (col("geom_OCC").isNotNull() & col("geom_SP").isNotNull())
    )
)

wide_df = wide_df.join(enverus_well_name, on="uwi_10", how="left")


#display(wide_df)

wide_df.write.format("delta").mode("overwrite").saveAsTable("geodata.gold.well_surface_locations")




In [0]:
%sql
select * from geodata.gold.well_surface_locations

## Filter by the largest variance
Using SQL `OR` operator lets us pick all wells where any vendor's Lat/Lon point exceeds a threshold. Switch to the `AND` operator to see where they _really_ don't match.

## Which one is correct?
Excellent question!

* Most newer wells are verified with GPS. These may or may not agree with regulatory agencies or vendors.
* Algorithmically defined Latitude/Longitudes based on satellite or aerial photos provide the next-best option. 
* The oldest well locations are inferred from Township/Range (or something more archaic).

In other words, "correctness" needs to account for courthouse records, subsurface mineral rights and lawyers. Newer wells generally have more geospatially accurate and agreed-upon location data, but all those varying sources of truth over time can lead to some geospatial confusion.

In [0]:
from pyspark.sql.functions import col

df = spark.table("geodata.gold.well_surface_locations")

DISTANCE_THRESHOLD = 100

# Filter for rows where at least one pairwise distance exceeds the threshold
df_filtered = df.filter(
    (col("distance_env_occ") > DISTANCE_THRESHOLD) |
    (col("distance_env_sp") > DISTANCE_THRESHOLD) |
    (col("distance_occ_sp") > DISTANCE_THRESHOLD)
)

display(df_filtered)